<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2025/blob/main/Seminars/BioInf/S9_Boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Основные методы ML

### Семинар 9: Boosting модели. Градиентный бустинг, XGBoost, CatBoost. Explainability модели. SHAP

#### Применение boosting моделей к реальным данным

In [ ]:
!pip install shap

### Задача 1: SHAP Explainability model

В данном упражнении на датасете Breast cancer wisconsin следует продемонстрировать работу моделей градиентного бустинга, xgboost, lightgbm и catboost.

Импортируйте необходимые библиотеки указанные в лекции.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import shap
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Загрузите датасет

In [ ]:
X, y = datasets.load_breast_cancer(return_X_y=True, as_frame=True)
X.head()

Разделите данные на обучающую и тестовую выборки в соотношении 4:1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### **Обучение моделей**

Выберите две модели для обучения. Можно выбрать два метода бустинга, можно выбрать лес и бустинг

In [ ]:
forest = ensemble.RandomForestClassifier(max_depth=6, n_estimators=1000, random_state=0, n_jobs=-1)
forest.fit(X_train, y_train)

In [ ]:
y_train.head()

#### **Создание объясняющих SHAP моделей**

Создайте при помощи shap.TreeExplainer модели, объясняющие предсказания

In [ ]:
explainer = shap.TreeExplainer(forest)
shap_values_test = explainer.shap_values(X_test)
shap_values_test.shape

#### **Локальное объяснение**

Выберите пример из датасета и сравните объяснения предсказаний от выбранных моделей

In [ ]:
# j = np.random.randint(0, X_test.shape[0])
j = 65

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values_test[j, :, 1], X_test.iloc[j])

In [ ]:
y_train.mean()

**Пример на XGBoost**

In [ ]:
from xgboost import XGBClassifier

XGBmodel = XGBClassifier() # возьмем с дефолтными параметрами
XGBmodel.fit(X_train, y_train)

In [ ]:
explainerXGB = shap.TreeExplainer(XGBmodel)
shap_values_XGB_test = explainerXGB.shap_values(X_test)

In [ ]:
j = 65

In [ ]:
shap.initjs()
shap.force_plot(explainerXGB.expected_value, shap_values_XGB_test[j], X_test.iloc[j])

**Объяснение модели**

Теперь постройте `summary plot` для модели, который поможет объяснить предсказания модели в целом на датасете.

In [ ]:
shap.summary_plot(shap_values_test[:, :, 1], X_train, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values_XGB_test, X_train, plot_type="bar")

### Задача 2: Предсказание растворимости веществ, подбор параметров

Загрузим датасет [AqSolDB](https://www.nature.com/articles/s41597-019-0151-1), объединяющий информацию о растворимости химических веществ из разных источников.



In [ ]:
!wget  https://kodomo.fbb.msu.ru/FBB/year_20/ml/curated-solubility-dataset.csv

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("curated-solubility-dataset.csv")

In [ ]:
data.head()

В качестве Y используйте колонку **Solubility**

In [ ]:
y = data['Solubility']
y.head()

В качестве X - все колонки после Solubility

In [ ]:
X = data.iloc[:, 6:]
X.head()

Есть ли здесь колонки, которые требуется предварительно обработать? Какие? Проведите предобработку

In [ ]:
X.Group.value_counts()

In [ ]:
# SD - некоторая "уверенность" в предсказанном значении

In [ ]:
X.SD.value_counts()

In [ ]:
X_transformed = pd.get_dummies(X, columns = ["Group"])
X_transformed

Разбейте ваши данные на обучение и тест в отношении 4:1

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2,
                                                    random_state=777)

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)

In [ ]:
importance = pd.DataFrame({"features": X_train.columns, "coef": linreg.coef_})
importance.sort_values(by="coef", ascending=False)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train_sc, y_train)

In [ ]:
importance = pd.DataFrame({"features": X_train.columns, "coef_abs": np.abs(linreg.coef_)})
importance.sort_values(by="coef_abs", ascending=False)

In [ ]:
y_pred = linreg.predict(X_test)

In [ ]:
X_train.corr()